# TODOLIST

1) multy-query
2) bm25retriever
3) rag-fusion
4) chain of thouths (decomposition?)
5) "+" few shot
6) Hyde (не дорого ли?)
7) Query structuring for metadata filters?
8) raptor

In [1]:
# def multy_doc(files):
#     all_chunks = []
#     for uploaded_file in files:
#         if uploaded_file.name.endswith('.pdf'):
#             loader = PyPDFLoader(uploaded_file)
#             documents = loader.load()
#         elif uploaded_file.name.endswith('.docx'):
#             loader = Docx2txtLoader(uploaded_file)
#             documents = loader.load()
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=300)
#         chunks = text_splitter.split_documents(documents)
#         all_chunks.extend(chunks)
#     return all_chunks

In [2]:
# docs =[]
# with tempfile.TemporaryDirectory() as tmpdir:

#     upfiles = st.file_uploader("Выберите файлы", type=['pdf'], accept_multiple_files= True)
#     if upfiles:
#         file_paths = []
    
#         for uploaded in upfiles:
#             file_path = os.path.join(tmpdir, uploaded.nameb)
#             with open(file_path, 'wb') as f:
#                 f.write(uploaded.getbuffer())
#             file_paths.append(file_paths)

#         selected_file = st.selectbox("Выберите файлы", options=[os.path.basename(path) for path in file_paths])
#         if selected_file:
#             selected_file_path = [path for path in file_path if os.path.basename(path) == selected_file][0]
#             if selected_file.endswith(".pdf"):
#                 loader = PyPDFLoader(selected_file_path)
#                 documents = loader.load()
#                 text_splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=450)
#                 splitted_data = text_splitter.split_documents(documents)
#                 docs.extend(splitted_data)

# st.write(docs)

In [ ]:
! pip install ragas

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS

In [3]:
import os
os.environ['CURL_CA_BUNDLE'] = ''

In [6]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    )

C:\Users\Grisha\AppData\Local\Temp\ipykernel_2488\720198466.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(
c:\Users\Grisha\Desktop\RAGWORK\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
file_path1 = 'DB_small'
file_path2 = 'DB_big'

In [8]:
vector_store_small_clear = FAISS.load_local(folder_path=file_path1, embeddings=embedding, allow_dangerous_deserialization= True)
vector_store_big_clear = FAISS.load_local(folder_path=file_path2, embeddings=embedding, allow_dangerous_deserialization= True)

In [26]:
vector_store_big_clear.docstore._dict

{'a722cf13-d319-4489-b112-5820defccbec': Document(metadata={'source': '/kaggle/input/papers/8882596886749806284.pdf'}, page_content='Race to the top Assessing and accelerating drone readiness in the UK the G7 and other leading nations Copyright 2023 GSMA Intelligence The GSMA is a global organisation unifying the mobile ecosystem to discover develop and deliver innovation foundational to positive business environments and societal change Our vision is to unlock the full power of connectivity so that people industry and society thrive Representing mobile operators and organisations across the mobile ecosystem and adjacent industries the GSMA delivers for its members across three broad pillars Connectivity for Good Industry Services and Solutions and Outreach This activity includes advancing policy tackling todays biggest societal challenges underpinning the technology and interoperability that make mobile work and providing the worlds largest platform to convene the mobile ecosystem at 

In [27]:
vector_store_small_clear.docstore._dict

{'269b2d9d-d968-44f5-9893-fca9f1c76593': Document(metadata={'source': '/kaggle/input/paper2/7083090788703859399.pdf'}, page_content='1 Executive Summary Silicon Valley and Wall Street love to hype articial intelligence AI The more its used they say the more diseases well cure the fewer errors well makeand the lower emissions will go Googles AI subsidiary DeepMind claimed advances in AGI articial generative intelligence research will supercharge societys ability to tackle and manage climate change1 At COP28 last year Google released a new report proclaiming 510 of global greenhouse gas emissions could be mitigated by the use of AI2 But there are two signicant and immediate dangers posed by AI that are much less discussed 1 the vast increase in energy and water consumption required by AI systems like ChatGPT and 2 the threat of AI turbocharging disinformationon a topic already rife with antiscience lies3 and funded4 by fossil fuel companies5 and their networks6 First the industry now ack

In [28]:
print(len(vector_store_big_clear.docstore._dict), len(vector_store_small_clear.docstore._dict))

134058 166337


In [29]:
vector_store_big_clear.merge_from(vector_store_small_clear)

Оценка качества ретриверов на грязных и почищенных данных. Тестировать будем для RAG без памяти, для экономии токенов

In [7]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
)

from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

In [108]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['CURL_CA_BUNDLE'] = ''

Базовый пайплайн из коробки

In [60]:
credentials_test = os.getenv('credentials_test')

In [126]:
credentials_test = os.getenv('credentials_test')
llm = GigaChat(credentials=credentials_test, verify_ssl_certs=False)#auth_url = 'https://sm-auth-sd.prom-88-89-apps.ocp-geo.ocp.sigma.sbrf.ru/api/v2/oauth'

prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя.
Используй при этом только информацию из контекста. Отвечай на русском языке.
Контекст: {context}
Вопрос: {input}
Ответ:''')
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    )

c:\Users\Grisha\Desktop\RAGWORK\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [127]:
# несколько доков, но для удобства возьмём 1 большой
# documents = []
# c=0
# for file in os.listdir('test'):
#         loader = PyPDFLoader('test' + '/'+ file)
#         docs = loader.load()
#         c+=1
#         print(c)

#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#         splits = text_splitter.split_documents(docs)
#         documents.extend(splits)


In [128]:
loader = PyPDFLoader('ai-hr.pdf')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [129]:
len(documents)

131

In [130]:
vectorstore = FAISS.from_documents(documents=documents, embedding=embedding)

In [131]:
! pip install pdfminer.six


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [132]:
from langchain.document_loaders import PDFMinerLoader

In [133]:
import re

def clean_text_from_pdf(docum):
    for chunk in range(len(docum)):
        text = doc[chunk].page_content
        text = re.sub(r'http[s]?://\S+|www\.\S+', '', text)  
    # Убираем изображения, которые могут быть представлены как ссылки или текст
        text = re.sub(r'\[.*?\]\(.*?\)|(?:download(?:[^\n]*?))|(?:image(?:[^\n]*?))', '', text)
    # Убираем строки, представляющие собой таблицы (например, строка с символами "|")
        text = re.sub(r'(^|\n)\|.*?\|(\n|$)', '', text) # убрать строки с разделителем `|
    # Удаляем специальные символы, оставляя только буквы, цифры и пробелы
        text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', text)
    # Удаляем лишние пробелы и пробелы в начале и в конце строки
        text = re.sub(r'\s+', ' ', text).strip()
        docum[chunk].page_content = text
    return doc

In [135]:
loader = PDFMinerLoader('ai-hr.pdf',concatenate_pages=True)
doc = loader.load()
doc = clean_text_from_pdf(doc)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=300)
clean_data = text_splitter.split_documents(doc)


In [137]:
len(clean_data)

150

In [138]:

vectorstore_clean = FAISS.from_documents(documents=clean_data, embedding=embedding)

In [93]:
# Сожрало все токены по итогу бесполезно. В граунд труз и вопрос пишет про китайские стены когда контекст про разрыв в уровнее ИИ разных стран
# from ragas.testset.generator import TestsetGenerator
# from ragas.testset.evolutions import simple, reasoning, multi_context

# # generator 
# generator_llm = llm
# critic_llm = llm
# embeddings = embedding

# generator = TestsetGenerator.from_langchain(
#     generator_llm,
#     critic_llm,
#     embeddings,
#     # docstore=vectorstore
# )

# testset = generator.generate_with_langchain_docs(documents, test_size=1, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

In [94]:
testset

TestDataset(test_data=[DataRow(question='What were some ancient fortifications used for and how did they contribute to the security and defense of their respective civilizations?', contexts=['ловеческий потенциал, остаются фундаментальной проблемой во многих разви-\nвающихся странах, в частности в наименее развитых странах, что может спо-\nсобствовать рас ширению разрывов в области искусственного интеллекта и дру-\nгих цифровых технологий, в том числе различий в степени готовности к эффек-\nтивному использованию систем искусственного интеллекта,  \n подчеркивая  необходимость и неотложность преодоления разрывов в обл а-\nсти искусственного интеллекта и других цифровых технологий между странами \nи внутри стран, а также особо отмечая необходимость обеспечения качествен-\nного доступа к преимуществам искусственного интеллекта, и в этой связи под-\nчеркивая важность обеспечения надл ежащего государственного и частного фи-\nнансирования, мобилизации ориентированных на практические действия

In [95]:
# df = testset.to_pandas()

In [96]:
# df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What were some ancient fortifications used for...,"[ловеческий потенциал, остаются фундаментально...",Ancient fortifications were used for various p...,simple,"[{'source': 'paper2.pdf', 'page': 2}]",True


In [88]:
# AAAA = df.copy()

In [92]:
# AAAA[['question','contexts','ground_truth']]

,question,contexts,ground_truth
0,What were some ancient fortifications used for...,"[ловеческий потенциал, остаются фундаментально...",Ancient fortifications were used for various p...


In [98]:
from datasets import load_dataset

# loading the V2 dataset
amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2", trust_remote_code=True)
amnesty_qa

Repo card metadata block was not found. Setting CardData to empty.
Generating eval split: 20 examples [00:00, 217.39 examples/s]


DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 20
    })
})

In [118]:
from datasets import Dataset 

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on January 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The Super Bowl....season since 1966,','replacing the NFL...in February.'], 
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)

In [120]:
dataset.to_pandas()

,question,answer,contexts,ground_truth
0,When was the first super bowl?,"The first superbowl was held on January 15, 1967","[The Super Bowl....season since 1966,, replaci...","The first superbowl was held on January 15, 1967"
1,Who won the most super bowls?,The most super bowls have been won by The New ...,"[The Green Bay Packers...Green Bay, Wisconsin....",The New England Patriots have won the Super Bo...


In [121]:
results = evaluate(dataset=dataset, metrics=[faithfulness,
    answer_relevancy,
    context_precision,
    context_recall], llm=llm, embeddings=embedding)

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
AUTHENTICATION ERROR
Evaluating: 100%|██████████| 8/8 [00:06<00:00,  1.21it/s]


In [122]:
results

{'faithfulness': 1.0000, 'answer_relevancy': 0.9116, 'context_precision': 0.5000, 'context_recall': 1.0000}

In [123]:
# from ragas.integrations.langchain import EvaluatorChain
# from ragas.metrics import (
#     faithfulness,
#     answer_relevancy,
#     context_precision,
#     context_recall,
# )

# # create evaluation chains
# faithfulness_chain = EvaluatorChain(metric=faithfulness)
# answer_rel_chain = EvaluatorChain(metric=answer_relevancy)
# context_rel_chain = EvaluatorChain(metric=context_precision)
# context_recall_chain = EvaluatorChain(metric=context_recall)

In [124]:
# from ragas import evaluate

# result = evaluate(
#     amnesty_qa["eval"],
#     metrics=[
#         context_precision,
#         faithfulness,
#         answer_relevancy,
#         context_recall,
#     ],
# )

# result

In [38]:

vectorstore = FAISS.from_documents(documents=documents, embedding=embedding)
retriever = vectorstore.as_retriever()

document_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=prompt
        )
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [40]:
question = "Развитие экономики"
resp = retrieval_chain.invoke({'input': question})
resp

{'input': 'Развитие экономики',
 'context': [Document(metadata={'source': 'test/paper.pdf', 'page': 7}, page_content='общего назначения становится основным драйве-\nром экономического развития, повышает произво-\nдительность труда в ныне существующих сферах \nдеятельности, способствует формированию новых \nотраслей и рынков, качественно меняет сферу \nуслуг.\nИндекс готовности отраслей экономики к внедре-\nнию ИИ (далее − Индекс ИИ), впервые разрабо-\nтанный и использованный в 2021 году2, является \nдейственным инструментом для управления про-\nцессами разработки и использования технологий \nИИ в сферах деятельности Российской Федерации.\n Регулярное применение Индекса ИИ позволяет не \nтолько отслеживать эффективность использования \nтехнологий ИИ в различных сферах деятельности \nи оценивать имеющиеся для этого благоприятные \nусловия, но и управлять изменениями для достиже-\nния целей Национальной стратегии развития ИИ.\nРазработанная в 2021 году концептуальная схема \nи методология

1400 токенов

In [133]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="вопрос",
    description="вопрос по контексту."
)

question_response_schemas = [
    question_schema,
]

In [134]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [135]:
question_generation_llm = GigaChat(credentials=credentials_test, verify_ssl_certs=False)

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [136]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
Вы профессиональный аналитик документов. Для каждого контекста создайте вопрос, специфичный для контекста. Не задавайте общие вопросы. Делайте всё на русском языке.

вопрос: вопрос по контексту.

Format the output as JSON with the following keys:
вопрос

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=docs[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
output_dict = question_output_parser.parse(response.content)

In [137]:
for k, v in output_dict.items():
  print(k)
  print(v)

вопрос
Какой документ был предоставлен?
context
{'page_content': '', 'metadata': {'source': 'test/paper.pdf', 'page': 0}}


In [110]:
! pip install -q -U tqdm


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [138]:
from tqdm import tqdm

qac_triples = []

for text in tqdm(docs[:10]):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})
  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue
  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 10/10 [00:08<00:00,  1.16it/s]


In [139]:
qac_triples[5]

{'вопрос': 'Какие основные результаты были достигнуты в секторе ИТ в 2023 году?',
 'context': Document(metadata={'source': 'test/paper.pdf', 'page': 5}, page_content='Оглавление\nВВЕДЕНИЕ ...............................................................................................................................................\n1. ОСНОВНЫЕ РЕЗУЛЬТАТЫ 2023 ГОДА ...................................................................................................\n2. РЕЗУЛЬТАТЫ 2023 ГОДА ДЛЯ СФЕР ДЕЯТЕЛЬНОСТИ ...........................................................................\n2.1. ФИНАНСОВЫЕ УСЛУГИ ......................................................................................................................\n2.2. СЕКТОР ИКТ ......................................................................................................................................\n2.3. ЗДРАВООХРАНЕНИЕ ..............................................................................................

In [141]:
answer_generation_llm = GigaChat(credentials=credentials_test, verify_ssl_certs=False)

answer_schema = ResponseSchema(
    name="ответ",
    description="ответ на вопрос"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\

Вы профессиональный аналитик документов. Для каждого вопроса и контекста придумайте ответ. Делайте всё на русском языке.

ответ: ответ по контексту.

Format the output as JSON with the following keys:
ответ

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["вопрос"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

In [142]:
for k, v in output_dict.items():
  print(k)
  print(v)

ответ
На основании представленного документа можно сделать вывод о том, что...


In [144]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["вопрос"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  triple["ответ"] = output_dict["ответ"]

100%|██████████| 9/9 [00:10<00:00,  1.15s/it]


In [52]:
! pip install -q -U datasets


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [149]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triples)


In [150]:
ground_truth_qac_set

,вопрос,context,ответ
0,Какой вывод можно сделать на основе представле...,page_content='' metadata={'source': 'test/pape...,На основании представленного документа можно с...
1,Какой основной вывод можно сделать из представ...,page_content='' metadata={'source': 'test/pape...,"Основной вывод, который можно сделать из предс..."
2,Какие отрасли экономики Российской Федерации и...,"page_content='Москва, 2023ПРАВИТЕЛЬСТВО РОССИЙ...",Ответ по контексту
3,Какие приоритетные отрасли экономики Российско...,page_content='УДК 004.8 + 338\nИндекс готовнос...,В данном докладе исследованы приоритетные отра...
4,Какие организации являются партнерами исследов...,page_content='Партнеры исследования\nЦентр ком...,"Центр компетенций федерального проекта ""Искусс..."
5,Какие основные результаты были достигнуты в се...,page_content='Оглавление\nВВЕДЕНИЕ ..............,Основные результаты сектора ИТ в 2023 году не ...
6,Какие отрасли являются приоритетными для внедр...,page_content='ИНДЕКС ГОТОВНОСТИ ПРИОРИТЕТНЫХ О...,Ответ по контексту.
7,Какие отрасли считаются приоритетными для внед...,page_content='ИНДЕКС ГОТОВНОСТИ ПРИОРИТЕТНЫХ О...,Ответ по контексту.
8,Какие отрасли экономики и сектора социальной с...,page_content='4\nОСНОВНЫЕ РЕЗУЛЬТАТЫ 2023 ГОДА...,"Финансовые услуги, сектор ИКТ, здравоохранение"


In [151]:

ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"ответ" : "ground_truth"})


In [152]:


eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [153]:
eval_dataset

Dataset({
    features: ['вопрос', 'context', 'ground_truth'],
    num_rows: 9
})

Как мы можем заметить половина вопросов мусорные

In [67]:
! pip install ragas

  Obtaining dependency information for ragas from https://files.pythonhosted.org/packages/5a/67/754ee2191991ce20dde5bfea5547e35f600773d1eb16c8fc00082ba4f02b/ragas-0.1.20-py3-none-any.whl.metadata
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/bf/4b/48ca098cb580c099b5058bf62c4cb5e90ca6130fa43ef4df27088536245b/tiktoken-0.7.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for langchain-core<0.3 from https://files.pythonhosted.org/packages/bc/02/2b2cf9550cee1a7ffa42fe60c55e2d0e7d397535609b42562611fb40e78d/langchain_core-0.2.41-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-openai from https://files.pythonhosted.org/packages/47/62/de45798e51dc030f092c216bd0be93246c73fce2f612ef9a60fa032538ce/langchain_openai-0.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for openai>1 from https://files.pythonhosted.org/packages/ee/49/7baf2537d7eedb162c8212e973bd69fe7233b463118e469bdb1bb07506f6


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [155]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)

from ragas.metrics.critique import harmfulness
from ragas import evaluate

def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["вопрос"]})
    rag_dataset.append(
        {"question" : row["вопрос"],
         "answer" : answer["response"].content,
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases